# Пункт - 1 - Обучение модели (1 балл)

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

Шаблон кода по настройке валидации и оценке качества:
```
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=12341234)

cv_predictions = []
cv_target = []
for idx, (train_idx, val_idx) in enumerate(cv.split(X=df, y=df['Survived'])):
    train = df.iloc[train_idx]
    val = df.iloc[val_idx]

    y_train = train['Survived']
    y_val = val['Survived']

    x_train = train.drop('Survived', axis=1)
    x_val = val.drop('Survived', axis=1)

    model = LogisticRegression()

    model.fit(X=x_train, y=y_train)
    pred_train = model.predict_proba(x_train)[:, 1]
    pred_val = model.predict_proba(x_val)[:, 1]
    cv_predictions.append((pred_train, pred_val))

    cv_target.append((y_train, y_val))

pred_train = np.hstack([i[0] for i in cv_predictions])
pred_val = np.hstack([i[1] for i in cv_predictions])

cv_target_train = np.hstack([i[0] for i in cv_target])
cv_target_val = np.hstack([i[1] for i in cv_target])

model_quality = roc_auc_score(y_true=cv_target_val, y_pred=pred_val)
```

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ

# Пункт - 2 - Оценка качества оптимальной модели (1 балл)
- Построение и анализ ROC и PR кривых
- Расчет precision, recall, f1-score при пороге классификации 0.5
- Подбор порога классификации и расчет precision, recall, f1-score для этого порога

## Решение
В этом блоке необходимо написать код. Можно создать столько ячеек с кодом, сколько нужно для выполнения задания

In [ ]:
# Здесь код решения

## Ответ
В этом блоке необходимо вывести ответ для этой части задания. Ответ должен описаться на написанный код, построенные визуализации и проведенные расчеты. Если необходимо, то можно записать ответ в нескольких ячейках.

In [ ]:
# Здесь код подготовки ответа и сам ответ